In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib import cm
import xarray as xr
import pandas as pd
import datetime
import warnings # Silence the warnings from SHARPpy
warnings.filterwarnings("ignore")
from scipy import interpolate
from scipy.ndimage import gaussian_filter
import scipy
import scipy.stats
import math
import cmweather
import matplotlib.patches as mpatches
import glob
import os
from metpy.units import units
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 20
from matplotlib.ticker import FormatStrFormatter
import pingouin as pg
import shapely
import shapely.plotting
import shapely.wkt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

In [ ]:
d = {}
d_full = {}
d_full2 = {}


lower_shear = [6, 8 , 10]
upper_shear = [31, 33, 35]

path = '/data/keeling/a/melinda3/gpm/data_dir/'
for li in lower_shear:
    for ui in upper_shear:
        hash = 'final_tracking_algo_hodo_test_'+str(li)+'_'+str(ui)+'_1m_dss_full.csv'
        file_in = pd.read_csv(path+hash)
        file_in = file_in.replace(-9999, np.nan)
        d_full["area{0}_{1}_dss".format(li, ui)] = file_in
        
        hash = 'final_tracking_algo_hodo_test_'+str(li)+'_'+str(ui)+'_1m_lss_full.csv'
        file_in = pd.read_csv(path+hash)
        file_in = file_in.replace(-9999, np.nan)
        d_full["area{0}_{1}_lss".format(li, ui)] = file_in

        hash = 'final_tracking_algo_hodo_test_'+str(li)+'_'+str(ui)+'_1m_hss_full.csv'
        file_in = pd.read_csv(path+hash)
        file_in = file_in.replace(-9999, np.nan)
        d_full["area{0}_{1}_hss".format(li, ui)] = file_in

        hash = 'final_tracking_algo_hodo_test_'+str(li)+'_'+str(ui)+'_1m_lss15_full.csv'
        file_in = pd.read_csv(path+hash)
        file_in = file_in.replace(-9999, np.nan)
        d_full["area{0}_{1}_l15".format(li, ui)] = file_in

        hash = 'final_tracking_algo_hodo_test_'+str(li)+'_'+str(ui)+'_1m_hss15_full.csv'
        file_in = pd.read_csv(path+hash)
        file_in = file_in.replace(-9999, np.nan)
        d_full["area{0}_{1}_h15".format(li, ui)] = file_in


d = d_full

## Heatmaps (Fig 3)

In [ ]:
uca_con = np.zeros(len(d))
ota_con = np.zeros(len(d))
bshear_con = np.zeros(len(d))
hodol_con = np.zeros(len(d))
hodor_con = np.zeros(len(d))
cv_con = np.zeros(len(d))
n2_con = np.zeros(len(d))
srh_con = np.zeros(len(d))
otd_con = np.zeros(len(d))
upper_area_con = np.zeros(len(d))
upper_velo_con = np.zeros(len(d))
lower_area_con = np.zeros(len(d))
lower_velo_con = np.zeros(len(d))
el_velo_con = np.zeros(len(d))


n = 0
for key in d:
    data = d[key]
    data = data[data.time > data.time_split.iloc[0]]
    uca_bshear = data['bshear06'].iloc[0]
    n2 = data['n2'].iloc[0]
    srh = data['srh03'].iloc[0]

    core_data = data[(data.z > 4) & (data.z < 8)]
    ot_data = data[(data.z > 13.3) & (data.z < 13.4)]
    lower_data = data[(data.z > 2.5) & (data.z <= 4)] 
    upper_data = data[(data.z >= 8) & (data.z <=13.3)]
    otd_data = data[(data.z > 13.375)]
    otd_grouped = otd_data.groupby("time", dropna = True)
    grouped_core_data = core_data.groupby("time", dropna = True)
    grouped_upper_data = upper_data.groupby('time', dropna =True)
    grouped_lower_data = lower_data.groupby('time', dropna = True)
    grouped_ot_data = ot_data.groupby("time", dropna=True)

    uca_area = grouped_core_data.mean('polygon_med_area').max()['polygon_med_area']
    core_velo = grouped_core_data.mean('max_velo').max()['max_velo']
    ota_area = grouped_ot_data.mean('polygon_med_area').max()['polygon_med_area']
    lower_area = grouped_lower_data.mean('polygon_med_area').max()['polygon_med_area']
    upper_area = grouped_upper_data.mean('polygon_med_area').max()['polygon_med_area']
    upper_velo = grouped_upper_data.mean('max_velo').max()['max_velo'] 
    lower_velo = grouped_lower_data.mean('max_velo').max()['max_velo']
    otd = otd_grouped.max('z').max()['z']
    otd = otd - 13.375
    el_velo = grouped_ot_data.mean('max_velo').max()['max_velo']

    if "6" in key:
        hodo_rad_key = np.int64(key[4])
        hodo_len_key = np.int64(key[6:8])
    elif "8" in key:
        hodo_rad_key = np.int64(key[4])
        hodo_len_key = np.int64(key[6:8])
    else:
        hodo_rad_key = np.int64(key[4:6])
        hodo_len_key = np.int64(key[7:9])

    uca_con[n] = uca_area * .5 * .5
    ota_con[n] = ota_area * .5 * .5
    bshear_con[n] = uca_bshear
    cv_con[n] = core_velo
    hodor_con[n] = hodo_rad_key
    hodol_con[n] = hodo_len_key
    n2_con[n] = n2
    srh_con[n] = srh
    upper_area_con[n] = upper_area * .5 * .5
    lower_area_con[n] = lower_area * .5 * .5
    el_velo_con[n] = el_velo
    otd_con[n] = otd
    lower_velo_con[n] = lower_velo
    upper_velo_con[n] = upper_velo

    n+=1

dict = {'uca':uca_con, 'ota':ota_con, 'bshear':bshear_con,\
        'core_velo':cv_con, 'hodo_r':hodor_con, 'hodo_l':hodol_con,\
        'lca':lower_area_con, 'upca':upper_area_con, 'n2':n2_con,\
        'srh':srh_con, 'lvel':lower_area_con, 'upvel':upper_velo_con,\
        'el_vel':el_velo_con, 'otd':otd_con}

data_dict = pd.DataFrame(dict)

In [ ]:
default_winds = data_dict[(data_dict.hodo_r == 8)  & (data_dict.hodo_l == 33)]
def_dss = default_winds.iloc[0]

uca_def = def_dss.uca
ota_def = def_dss.ota
otd_def = def_dss.otd

#Get percent changes for OTA/UCA/OTD relative to the 8,33,DSS profile

uca_pct_change = ((data_dict['uca'] - uca_def) / uca_def)  * 100
ota_pct_change = ((data_dict['ota'] - ota_def) / ota_def)  * 100 
otd_pct_change = ((data_dict['otd'] - otd_def) / otd_def)  * 100

data_dict['uca_pct'] = uca_pct_change
data_dict['ota_pct'] = ota_pct_change
data_dict['otd_pct'] = otd_pct_change

In [ ]:
def get_heatmaps(data_in, column_list, feature_interest):
    data_sub = data_in[column_list]
    ota_min = data_sub[feature_interest].min()
    ota_max = data_sub[feature_interest].max()


    column_list_stub_hr = [feature_interest, 'n2', 'hodo_r']
    column_list_stub_hl = [feature_interest, 'n2', 'hodo_l']


    #Subset the data and make plots into features of interest for different HL values
    data_31 = data_sub[data_sub.hodo_l == 31]
    data_31 = data_31[column_list_stub_hr]
    data_group31 = data_31.groupby(['n2', 'hodo_r']).max().unstack(level = 0)

    data_33 = data_sub[data_sub.hodo_l == 33]
    data_33 = data_33[column_list_stub_hr]
    data_group33 = data_33.groupby(['n2', 'hodo_r']).max().unstack(level = 0)

    data_35 = data_sub[data_sub.hodo_l == 35]
    data_35 = data_35[column_list_stub_hr]
    data_group35 = data_35.groupby(['n2', 'hodo_r']).max().unstack(level = 0)

    n2_vals = data_31.n2.unique()
    n2_vals = sorted(n2_vals)

    if feature_interest == 'otd':
        title_str = 'OTD'
        cbar_label = 'OTD (km)'
    elif feature_interest == 'uca':
        title_str = 'UCA'
        cbar_label = 'UCA ($km^{2}$)' 
    elif feature_interest == 'ota':
        title_str = 'OTA'
        cbar_label = 'OTA ($km^{2}$)'
    elif feature_interest == 'core_velo':
        title_str = '$W_{max,core}$'
        cbar_label = '$W_{max,core}$ (m$\cdot$s$^{-1}$)'
    elif feature_interest == 'lca':
        title_str = 'LCA'
        cbar_label = 'LCA ($km^{2}$)' 
    elif feature_interest == 'upca':
        title_str = 'UPCA'
        cbar_label = 'UPCA ($km^{2}$)'  
    elif feature_interest == 'otd_pct':
        title_str = 'OTD'
        cbar_label = 'Percent Change'
    elif feature_interest == 'ota_pct':
        title_str = 'OTA'
        cbar_label = 'Percent Change'
    elif feature_interest == 'uca_pct':
        title_str = 'UCA'
        cbar_label = 'Percent Change'
    else: 
        return "Bad Title String Entry"
    

    fig1, (ax1, ax2, ax3) = plt.subplots(nrows = 1, ncols=3, figsize = (26,8))
    p1 = sns.heatmap(data_group31, annot = True, linewidth= 0.5, \
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax1, cmap = 'seismic')
    ax1.set_title(title_str+ ' Heatmap, HL = 31 m$\cdot$s$^{-1}$')
    ax1.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax1.set_ylabel('Hodograph Radius (m$\cdot$s$^{-1}$)')
    ax1.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)

    p2 = sns.heatmap(data_group33, annot = True, linewidth= 0.5,\
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax2, cmap = 'seismic')
    ax2.set_title(title_str+ ' Heatmap, HL = 33 m$\cdot$s$^{-1}$')
    ax2.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax2.set_ylabel('Hodograph Radius (m$\cdot$s$^{-1}$)')
    ax2.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)

    p3 = sns.heatmap(data_group35, annot = True, linewidth= 0.5, cbar_kws={'label':cbar_label},\
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax3, cmap = 'seismic')
    ax3.set_title(title_str+ ' Heatmap, HL = 35 m$\cdot$s$^{-1}$')
    ax3.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax3.set_ylabel('Hodograph Radius (m$\cdot$s$^{-1}$)')
    ax3.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)

    #Repeat this for different HR values

    data_6 = data_sub[data_sub.hodo_r == 6]
    data_6 = data_6[column_list_stub_hl]
    data_group6 = data_6.groupby(['n2', 'hodo_l']).max().unstack(level = 0)

    data_8 = data_sub[data_sub.hodo_r == 8]
    data_8 = data_8[column_list_stub_hl]
    data_group8 = data_8.groupby(['n2', 'hodo_l']).max().unstack(level = 0)

    data_10 = data_sub[data_sub.hodo_r == 10]
    data_10 = data_10[column_list_stub_hl]
    data_group10 = data_10.groupby(['n2', 'hodo_l']).max().unstack(level = 0)


    fig1, (ax1, ax2, ax3) = plt.subplots(nrows = 1, ncols=3, figsize = (26,8))
    p1 = sns.heatmap(data_group6, annot = True, linewidth= 0.5, \
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax1, cmap = 'seismic')
    ax1.set_title(title_str+ ' Heatmap, HR = 6 m$\cdot$s$^{-1}$')
    ax1.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax1.set_ylabel('Hodograph Length (m$\cdot$s$^{-1}$)')
    ax1.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)

    p2 = sns.heatmap(data_group8, annot = True, linewidth= 0.5,\
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax2, cmap = 'seismic')
    ax2.set_title(title_str + ' Heatmap, HR = 8 m$\cdot$s$^{-1}$')
    ax2.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax2.set_ylabel('Hodograph Length (m$\cdot$s$^{-1}$)')
    ax2.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)

    p3 = sns.heatmap(data_group10, annot = True, linewidth= 0.5, cbar_kws={'label':cbar_label},\
                    vmin = -30, vmax = 30, fmt=".02f",ax = ax3, cmap = 'seismic')
    ax3.set_title(title_str +' Heatmap, HR = 10 m$\cdot$s$^{-1}$')
    ax3.set_xlabel('$N^{2}$ (s$^{-2}$)')
    ax3.set_ylabel('Hodograph Length (m$\cdot$s$^{-1}$)')
    ax3.set_xticklabels(FormatStrFormatter('%0.2e').format_ticks(n2_vals), rotation = 45)




In [ ]:
#Make heatmaps for OTA
list = ['ota_pct', 'n2', 'hodo_r', 'hodo_l']
get_heatmaps(data_dict, list, 'ota_pct')

In [ ]:
#Make heatmaps for OTD
list = ['otd_pct', 'n2', 'hodo_r', 'hodo_l']
get_heatmaps(data_dict, list, list[0])

## Scatter Plots (Figs 2 and 4)

In [ ]:
f1 = plt.figure(figsize = (7,6))
#plt.title('${\Gamma}$ vs Max OTA', fontsize = 25)
plt.title('$N^{2}$ vs Max OTA', fontsize = 25)
#plt.xlabel('${\Gamma}$ ($^\circ$C$\cdot$$km^{-1}$)', fontsize = 20)
plt.xlabel('$N^{2}$ (s$^{-2}$)', fontsize = 15)
plt.ylabel('OTA ($km^{2}$)', fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.ticklabel_format(axis = 'x', style = 'scientific', scilimits=(2,4))
all_max_uca = []
all_max_ota = []
bshear6 = []
for key in d:
    bshear6.append(d[key]['bshear06'][5])

#Get the shear terciles
first_terc = np.percentile(bshear6, 33.3333)
second_terc = np.percentile(bshear6, 66.6666)

for key in d:
    if "lss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_lss = ota_slice.mean().max()
        max_ota_lss = ota_lss * .5 *.5

        uca_lss = uca_slice.max().max()
        max_uca_lss = uca_lss

        all_max_uca.append(max_uca_lss)
        all_max_ota.append(max_ota_lss)

        bshear_lss = d[key]['bshear06'][0]

        if bshear_lss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 6')
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 8')
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 10')
        elif first_terc < bshear_lss < second_terc:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 6)
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.5, markersize = 6)
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.5, markersize = 6)
        else:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 6)
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 1, markersize = 6)
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 1, markersize = 6)
    if "l15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_l15 = ota_slice.mean().max()
        max_ota_l15 = ota_l15 * .5 * .5

        uca_l15 = uca_slice.max().max()
        max_uca_l15 = uca_l15 

        all_max_uca.append(max_uca_l15)
        all_max_ota.append(max_ota_l15)

        bshear_l15 = d[key]['bshear06'][0]

        if bshear_l15 <= first_terc:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.25, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.25, markersize = 8)
        elif first_terc < bshear_l15 < second_terc:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.5, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.5, markersize = 8)
        else:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 1, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 1, markersize = 8)
    if "dss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_dss = ota_slice.mean().max()
        max_ota_dss = ota_dss * .5 * .5

        uca_dss = uca_slice.max().max()
        max_uca_dss = uca_dss 

        all_max_uca.append(max_uca_dss)
        all_max_ota.append(max_ota_dss)

        bshear_dss = d[key]['bshear06'][0]

        if bshear_dss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.25, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.25, markersize = 10)
        elif first_terc < bshear_dss < second_terc:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.5, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.5, markersize = 10)
        else:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 1, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 1, markersize = 10)
    if "h15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_h15 = ota_slice.mean().max()
        max_ota_h15 = ota_h15 * .5 * .5

        uca_h15 = uca_slice.max().max()
        max_uca_h15 = uca_h15

        all_max_uca.append(max_uca_h15)
        all_max_ota.append(max_ota_h15)

        bshear_h15 = d[key]['bshear06'][0]

        if bshear_h15 <= first_terc:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.25, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.25, markersize = 12)
        elif first_terc < bshear_h15 < second_terc:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.5, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.5, markersize = 12)
        else:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 1, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 1, markersize = 12)
    if "hss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_hss = ota_slice.mean().max()
        max_ota_hss = ota_hss * .5 * .5

        uca_hss = uca_slice.max().max()
        max_uca_hss = uca_hss

        all_max_uca.append(max_uca_hss)
        all_max_ota.append(max_ota_hss)

        bshear_hss = d[key]['bshear06'][0]
        
        if bshear_hss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.25, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.25, markersize = 14)
        elif first_terc < bshear_hss < second_terc:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.5, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.5, markersize = 14)
        else:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 1, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 1, markersize = 14)

uca_ota_model = scipy.stats.linregress(all_max_uca, all_max_ota)
r_value = uca_ota_model.rvalue
r2_val = r_value ** 2
b_val = uca_ota_model.intercept
m_val = uca_ota_model.slope
x_vals = np.arange(min(all_max_uca), max(all_max_uca), 0.000005)

y_vals = (x_vals*m_val) + b_val
plt.plot(x_vals, y_vals, ls = "--", color = 'k', \
    label = 'R = %0.2f\n'%r_value + "R$^2$ = %0.2f"%r2_val )
plt.legend(loc = 'best', fontsize = 12)


In [ ]:
f1 = plt.figure(figsize = (7,6))
plt.title('Max UCA vs Max OTA', fontsize = 25)
plt.xlabel('UCA ($km^{2}$)', fontsize = 20)
plt.ylabel('OTA ($km^{2}$)', fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
all_max_uca = []
all_max_ota = []
all_uca_6 = []
all_uca_8 = []
all_uca_10 = []
all_ota_6 = []
all_ota_8 = []
all_ota_10 = []
bshear6 = []
for key in d:
    bshear6.append(d[key]['bshear06'][0])

#Get the shear terciles
first_terc = np.percentile(bshear6, 33.3333)
second_terc = np.percentile(bshear6, 66.6666)

for key in d:
    if "lss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        bshear_lss = ld['bshear06'].iloc[0]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_lss = ota_slice.mean().max()
        max_ota_lss = ota_lss *.5 *.5

        uca_lss = uca_slice.mean().max()
        max_uca_lss = uca_lss * .5 *.5

        all_max_uca.append(max_uca_lss)
        all_max_ota.append(max_ota_lss)

        if bshear_lss <= first_terc:
            if "6" in key:
                all_uca_6.append(max_uca_lss)
                all_ota_6.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 6')
            elif "8" in key:
                all_uca_8.append(max_uca_lss)
                all_ota_8.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 8')
            else:
                all_uca_10.append(max_uca_lss)
                all_ota_10.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 10')
        elif first_terc < bshear_lss < second_terc:
            if "6" in key:
                all_uca_6.append(max_uca_lss)
                all_ota_6.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 6)
            elif "8" in key:
                all_uca_8.append(max_uca_lss)
                all_ota_8.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.5, markersize = 6)
            else:
                all_uca_10.append(max_uca_lss)
                all_ota_10.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.5, markersize = 6)
        else:
            if "6" in key:
                all_uca_6.append(max_uca_lss)
                all_ota_6.append(max_ota_lss)
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 6)
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 1, markersize = 6)
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 1, markersize = 6)
    if "l15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_l15 = ota_slice.mean().max()
        max_ota_l15 = ota_l15 *.5 *.5

        uca_l15 = uca_slice.mean().max()
        max_uca_l15 = uca_l15 * .5 *.5

        all_max_uca.append(max_uca_l15)
        all_max_ota.append(max_ota_l15)

        bshear_l15 = d[key]['bshear06'].iloc[0]
        if bshear_l15 <= first_terc:
            if "6" in key:
                all_uca_6.append(max_uca_l15)
                all_ota_6.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 8)
            elif "8" in key:
                all_uca_8.append(max_uca_l15)
                all_ota_8.append(max_ota_lss)
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.25, markersize = 8)
            else:
                all_uca_10.append(max_uca_l15)
                all_ota_10.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.25, markersize = 8)
        elif first_terc < bshear_l15 < second_terc:
            if "6" in key:
                all_uca_6.append(max_uca_l15)
                all_ota_6.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 8)
            elif "8" in key:
                all_uca_8.append(max_uca_l15)
                all_ota_8.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.5, markersize = 8)
            else:
                all_uca_10.append(max_uca_l15)
                all_ota_10.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.5, markersize = 8)
        else:
            if "6" in key:
                all_uca_6.append(max_uca_l15)
                all_ota_6.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 8)
            elif "8" in key:
                all_uca_8.append(max_uca_l15)
                all_ota_8.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 1, markersize = 8)
            else:
                all_uca_10.append(max_uca_l15)
                all_ota_10.append(max_ota_l15)
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 1, markersize = 8)
    if "dss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_dss = ota_slice.mean().max()
        max_ota_dss = ota_dss *.5 *.5

        uca_dss = uca_slice.mean().max()
        max_uca_dss = uca_dss * .5 *.5

        all_max_uca.append(max_uca_dss)
        all_max_ota.append(max_ota_dss)

        bshear_dss = d[key]['bshear06'].iloc[0]
        if bshear_dss <= first_terc:
            if "6" in key:
                all_uca_6.append(max_uca_dss)
                all_ota_6.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 10)
            elif "8" in key:
                all_uca_8.append(max_uca_dss)
                all_ota_8.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.25, markersize = 10)
            else:
                all_uca_10.append(max_uca_dss)
                all_ota_10.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.25, markersize = 10)
        elif first_terc < bshear_dss < second_terc:
            if "6" in key:
                all_uca_6.append(max_uca_dss)
                all_ota_6.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 10)
            elif "8" in key:
                all_uca_8.append(max_uca_dss)
                all_ota_8.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.5, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.5, markersize = 10)
        else:
            if "6" in key:
                all_uca_6.append(max_uca_dss)
                all_ota_6.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 10)
            elif "8" in key:
                all_uca_8.append(max_uca_dss)
                all_ota_8.append(max_ota_dss)
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 1, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 1, markersize = 10)
    if "h15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_h15 = ota_slice.mean().max()
        max_ota_h15 = ota_h15 *.5 *.5

        uca_h15 = uca_slice.mean().max()
        max_uca_h15 = uca_h15 * .5 *.5

        all_max_uca.append(max_uca_h15)
        all_max_ota.append(max_ota_h15)

        bshear_h15 = d[key]['bshear06'].iloc[0]
        if bshear_h15 <= first_terc:
            if "6" in key:
                all_uca_6.append(max_uca_h15)
                all_ota_6.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 12)
            elif "8" in key:
                all_uca_8.append(max_uca_h15)
                all_ota_8.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.25, markersize = 12)
            else:
                all_uca_10.append(max_uca_h15)
                all_ota_10.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.25, markersize = 12)
        elif first_terc < bshear_h15 < second_terc:
            if "6" in key:
                all_uca_6.append(max_uca_h15)
                all_ota_6.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 12)
            elif "8" in key:
                all_uca_8.append(max_uca_h15)
                all_ota_8.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.5, markersize = 12)
            else:
                all_uca_10.append(max_uca_h15)
                all_ota_10.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.5, markersize = 12)
        else:
            if "6" in key:
                all_uca_6.append(max_uca_h15)
                all_ota_6.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 12)
            elif "8" in key:
                all_uca_8.append(max_uca_h15)
                all_ota_8.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 1, markersize = 12)
            else:
                all_uca_10.append(max_uca_h15)
                all_ota_10.append(max_ota_h15)
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 1, markersize = 12)
    if "hss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z < 13.4) & (ld.z > 13.3)]
        uca_slice = uca_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_slice = ota_slice.groupby('time', dropna = True)['polygon_med_area']
        ota_hss = ota_slice.mean().max()
        max_ota_hss = ota_hss *.5 *.5

        uca_hss = uca_slice.mean().max()
        max_uca_hss = uca_hss * .5 *.5

        all_max_uca.append(max_uca_hss)
        all_max_ota.append(max_ota_hss)

        bshear_hss = d[key]['bshear06'].iloc[0]
        if bshear_hss <= first_terc:
            if "6" in key:
                all_uca_6.append(max_uca_hss)
                all_ota_6.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 14)
            elif "8" in key:
                all_uca_8.append(max_uca_hss)
                all_ota_8.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.25, markersize = 14)
            else:
                all_uca_10.append(max_uca_hss)
                all_ota_10.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.25, markersize = 14)
        elif first_terc < bshear_hss < second_terc:
            if "6" in key:
                all_uca_6.append(max_uca_hss)
                all_ota_6.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 14)
            elif "8" in key:
                all_uca_8.append(max_uca_hss)
                all_ota_8.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.5, markersize = 14)
            else:
                all_uca_10.append(max_uca_hss)
                all_ota_10.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.5, markersize = 14)
        else:
            if "6" in key:
                all_uca_6.append(max_uca_hss)
                all_ota_6.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 14)
            elif "8" in key:
                all_uca_8.append(max_uca_hss)
                all_ota_8.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 1, markersize = 14)
            else:
                all_uca_10.append(max_uca_hss)
                all_ota_10.append(max_ota_hss)
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 1, markersize = 14)

uca_ota_model = scipy.stats.linregress(all_max_uca, all_max_ota)
r_value = uca_ota_model.rvalue
r2_val = r_value ** 2
b_val = uca_ota_model.intercept
m_val = uca_ota_model.slope
x_vals = np.arange(min(all_max_uca), max(all_max_uca)+ 5, 5)
x_vals_6 = np.array([min(all_uca_6), max(all_uca_6)]) 
x_vals_8 = np.array([min(all_uca_8), max(all_uca_8)]) 
x_vals_10 = np.array([min(all_uca_10), max(all_uca_10)])

model_6 = scipy.stats.linregress(all_uca_6, all_ota_6)
model_8 = scipy.stats.linregress(all_uca_8, all_ota_8) 
model_10 = scipy.stats.linregress(all_uca_10, all_ota_10)  

m6 = model_6.slope
m8 = model_8.slope
m10 = model_10.slope
b6 = model_6.intercept
b8 = model_8.intercept
b10 = model_10.intercept

y6 = (x_vals_6*m6) + b6
y8 = (x_vals_8*m8) + b8
y10 = (x_vals_10*m10) + b10

y_vals = (x_vals*m_val) + b_val
plt.plot(x_vals, y_vals, ls = "--", color = 'k', \
    label = 'R = %0.2f\n'%r_value + "R$^2$ = %0.2f"%r2_val)
#plt.plot(x_vals_6, y6, ls = '--', color = 'fuchsia', zorder = -50)
#plt.plot(x_vals_8, y8, ls = '--', color = 'brown', zorder = -50)
#plt.plot(x_vals_10, y10, ls = '--', color = 'blue', zorder = -50)
plt.legend(loc = 'upper left', fontsize = 12)

In [ ]:
f1 = plt.figure(figsize = (7,6))
#plt.title('${\Gamma}$ vs Max OTD', fontsize = 25)
plt.title('$N^{2}$ vs Max OTD', fontsize = 25)
#plt.xlabel('${\Gamma}$ ($^\circ$C$\cdot$$km^{-1}$)', fontsize = 20)
plt.xlabel('$N^{2}$ (s$^{-2}$)', fontsize = 15)
plt.ylabel('Max OTD (km)', fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.ticklabel_format(axis = 'x', style = 'scientific', scilimits=(2,4))
all_max_uca = []
all_max_ota = []
bshear6 = []
for key in d:
    bshear6.append(d[key]['bshear06'][5])

#Get the shear terciles
first_terc = np.percentile(bshear6, 33.3333)
second_terc = np.percentile(bshear6, 66.6666)

for key in d:
    if "lss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z > 13.375)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['z']
        ota_lss = ota_slice.max().max()
        max_ota_lss = ota_lss - 13.375

        uca_lss = uca_slice.max().max()
        max_uca_lss = uca_lss

        all_max_uca.append(max_uca_lss)
        all_max_ota.append(max_ota_lss)

        bshear_lss = d[key]['bshear06'][0]

        if bshear_lss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 6')
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 8')
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.25, markersize = 6, \
                    label = 'HR = 10')
        elif first_terc < bshear_lss < second_terc:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 6)
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 0.5, markersize = 6)
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 0.5, markersize = 6)
        else:
            if "6" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 6)
            elif "8" in key:
                plt.plot(max_uca_lss, max_ota_lss, color = 'brown', marker = "o", alpha = 1, markersize = 6)
            else:
                plt.plot(max_uca_lss, max_ota_lss, color = 'blue', marker = "o", alpha = 1, markersize = 6)
    if "l15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z > 13.375)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['z']
        ota_l15 = ota_slice.max().max()
        max_ota_l15 = ota_l15  - 13.375

        uca_l15 = uca_slice.max().max()
        max_uca_l15 = uca_l15 

        all_max_uca.append(max_uca_l15)
        all_max_ota.append(max_ota_l15)

        bshear_l15 = d[key]['bshear06'][0]

        if bshear_l15 <= first_terc:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.25, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.25, markersize = 8)
        elif first_terc < bshear_l15 < second_terc:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 0.5, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 0.5, markersize = 8)
        else:
            if "6" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 8)
            elif "8" in key:
                plt.plot(max_uca_l15, max_ota_l15, color = 'brown', marker = "o", alpha = 1, markersize = 8)
            else:
                plt.plot(max_uca_l15, max_ota_l15, color = 'blue', marker = "o", alpha = 1, markersize = 8)
    if "dss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z > 13.375)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['z']
        ota_dss = ota_slice.max().max()
        max_ota_dss = ota_dss  - 13.375

        uca_dss = uca_slice.max().max()
        max_uca_dss = uca_dss 

        all_max_uca.append(max_uca_dss)
        all_max_ota.append(max_ota_dss)

        bshear_dss = d[key]['bshear06'][0]

        if bshear_dss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.25, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.25, markersize = 10)
        elif first_terc < bshear_dss < second_terc:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 0.5, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 0.5, markersize = 10)
        else:
            if "6" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 10)
            elif "8" in key:
                plt.plot(max_uca_dss, max_ota_dss, color = 'brown', marker = "o", alpha = 1, markersize = 10)
            else:
                plt.plot(max_uca_dss, max_ota_dss, color = 'blue', marker = "o", alpha = 1, markersize = 10)
    if "h15" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z > 13.375)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['z']
        ota_h15 = ota_slice.max().max()
        max_ota_h15 = ota_h15 - 13.375

        uca_h15 = uca_slice.max().max()
        max_uca_h15 = uca_h15

        all_max_uca.append(max_uca_h15)
        all_max_ota.append(max_ota_h15)

        bshear_h15 = d[key]['bshear06'][0]

        if bshear_h15 <= first_terc:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.25, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.25, markersize = 12)
        elif first_terc < bshear_h15 < second_terc:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 0.5, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 0.5, markersize = 12)
        else:
            if "6" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'fuchsia', marker = "o", alpha = 1, markersize = 12)
            elif "8" in key:
                plt.plot(max_uca_h15, max_ota_h15, color = 'brown', marker = "o", alpha = 1, markersize = 12)
            else:
                plt.plot(max_uca_h15, max_ota_h15, color = 'blue', marker = "o", alpha = 1, markersize = 12)
    if "hss" in key:
        ld = d[key]
        ld = ld[ld.time > ld.time_split.iloc[0]]
        uca_slice = ld[(ld.z < 8) & (ld.z >= 4)]
        ota_slice = ld[(ld.z > 13.375)]
        uca_slice = uca_slice.groupby('time', dropna = True)['n2']
        ota_slice = ota_slice.groupby('time', dropna = True)['z']
        ota_hss = ota_slice.max().max()
        max_ota_hss = ota_hss - 13.375

        uca_hss = uca_slice.max().max()
        max_uca_hss = uca_hss

        all_max_uca.append(max_uca_hss)
        all_max_ota.append(max_ota_hss)

        bshear_hss = d[key]['bshear06'][0]
        
        if bshear_hss <= first_terc:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.25, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.25, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.25, markersize = 14)
        elif first_terc < bshear_hss < second_terc:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 0.5, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 0.5, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 0.5, markersize = 14)
        else:
            if "6" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'fuchsia', marker = "o", alpha = 1, markersize = 14)
            elif "8" in key:
                plt.plot(max_uca_hss, max_ota_hss, color = 'brown', marker = "o", alpha = 1, markersize = 14)
            else:
                plt.plot(max_uca_hss, max_ota_hss, color = 'blue', marker = "o", alpha = 1, markersize = 14)

uca_ota_model = scipy.stats.linregress(all_max_uca, all_max_ota)
r_value = uca_ota_model.rvalue
r2_val = r_value ** 2
b_val = uca_ota_model.intercept
m_val = uca_ota_model.slope
x_vals = np.arange(min(all_max_uca), max(all_max_uca), 0.000005)

y_vals = (x_vals*m_val) + b_val
plt.plot(x_vals, y_vals, ls = "--", color = 'k', \
    label = 'R = %0.2f\n'%r_value + "R$^2$ = %0.2f"%r2_val )
plt.legend(loc = 'best', fontsize = 12)
